# Artificial Neural Network

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
df = pd.read_csv('Churn_Modelling.csv')
X = df.iloc[:, 3:13].values
y = df.iloc[:, 13].values

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le_geography = LabelEncoder()
X[:, 1] = le_geography.fit_transform(X[:, 1])
le_gender = LabelEncoder()
X[:, 2] = le_gender.fit_transform(X[:, 2])
ohe = OneHotEncoder()
X = ohe.fit_transform(X).toarray()
# avoid dummy variable trap
X = X[:, 1:]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

# Convert numpy arrays to PyTorch tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)


# Define the ANN architecture
class ANN(nn.Module):
    def __init__(self, input_dim):
        super(ANN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 6)
        self.layer2 = nn.Linear(6, 6)
        self.output = nn.Linear(6, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.output(x))
        return x

# Create ANN classifier
input_dim = X_train.shape[1] # Get the number of features, input dimension
classifier = ANN(input_dim)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(classifier.parameters())

# Training loop (assuming X_train and y_train are PyTorch tensors)
num_epochs = 10
batch_size = 10

for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]

        # Forward pass
        outputs = classifier(batch_X)
        loss = criterion(outputs, batch_y.unsqueeze(1))

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions
with torch.no_grad():
    y_pred = classifier(X_test)
    y_pred = (y_pred > 0.5).float()

# Calculate accuracy (assuming y_test is a PyTorch tensor)
accuracy = (y_pred == y_test.unsqueeze(1)).float().mean()
print(f'Accuracy: {accuracy.item():.4f}')

Epoch [1/10], Loss: 0.3902
Epoch [2/10], Loss: 0.3502
Epoch [3/10], Loss: 0.3456
Epoch [4/10], Loss: 0.3384
Epoch [5/10], Loss: 0.2867
Epoch [6/10], Loss: 0.1906
Epoch [7/10], Loss: 0.0669
Epoch [8/10], Loss: 0.0564
Epoch [9/10], Loss: 0.0450
Epoch [10/10], Loss: 0.0322
Accuracy: 0.8085


In [12]:
from sklearn.metrics import confusion_matrix
y_pred_np = y_pred.numpy().squeeze()
y_test_np = y_test.numpy()
print(confusion_matrix(y_test_np, y_pred_np))

[[1554   41]
 [ 342   63]]


In [14]:
accuracy_manual = (y_pred_np == y_test_np).mean()
print(f'Accuracy (manual calculation): {accuracy_manual:.4f}')

Accuracy (manual calculation): 0.8085
